<a href="https://colab.research.google.com/github/yifan-zhou19/Policy_migration/blob/master/Uni_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  This file demonstrates the migration process using the video dataset.

Input data: the ABAC policies


In [2]:
#define ABAC policy (university)
#define ABAC policy (university samples)
U=[{'uid':'applicant1','position':'applicant'},
 {'uid':'csStu1','position':'student','udepartment':'cs','crsTaken':['cs101'],'crsTaught':['cs601']},
 {'uid':'csStu2','position':'student','udepartment':'cs','crsTaken':['cs101','cs601']},
 {'uid':'csFac1','position':'faculty','udepartment':'cs','crsTaught':['cs101']},
 {'uid':'csFac2','position':'faculty','udepartment':'cs','crsTaught':['cs601']},
 {'uid':'csChair','isChair':True,'udepartment':'cs'},
 {'uid':'eeFac1','position':'faculty','udepartment':'ee','crsTaught':['ee101']},
 {'uid':'eeFac2','position':'faculty','udepartment':'ee','crsTaught':['ee601']},
 {'uid':'eeChair','isChair':True,'udepartment':'ee'},
 {'uid':'registrar1','position':'staff','udepartment':'registrar'},
 {'uid':'admissions1','position':'staff','udepartment':'admissions'},
]
R=[{'rid':'application1','type':'application','student':'applicant1'},
 {'rid':'csStu1application','type':'application','student':'csStu1'},
 {'rid':'csStu2application','type':'application','student':'csStu2'},
 {'rid':'eeStu1application','type':'application','student':'eeStu1'},
 {'rid':'eeStu2application','type':'application','student':'eeStu2'},
 {'rid':'cs101gradebook','type':'gradebook','rdepartment':'cs','crs':'cs101'},
 {'rid':'cs601gradebook','type':'gradebook','rdepartment':'cs','crs':'cs601'},
 {'rid':'ee101gradebook','type':'gradebook','rdepartment':'ee','crs':'ee101'},
 {'rid':'ee601gradebook','type':'gradebook','rdepartment':'ee','crs':'ee601'},
 {'rid':'cs101roster','type':'roster','rdepartment':'cs','crs':'cs101'},
 {'rid':'cs601roster','type':'roster','rdepartment':'cs','crs':'cs601'},
 {'rid':'ee101roster','type':'roster','rdepartment':'ee','crs':'ee101'},
 {'rid':'ee601roster','type':'roster','rdepartment':'ee','crs':'ee601'},
 {'rid':'csStu1trans','type':'transcript','rdepartment':'cs','student':'csStu1'},
 {'rid':'csStu2trans','type':'transcript','rdepartment':'cs','student':'csStu2'},
 {'rid':'eeStu1trans','type':'transcript','rdepartment':'ee','student':'eeStu1'},
 {'rid':'eeStu2trans','type':'transcript','rdepartment':'ee','student':'eeStu2'},

]
UA=['position','udepartment','crsTaken','crsTaught','isChair','uid']
RA=['type','student','rdepartment', 'crs','rid']
O=['readMyScores','addScore', 'readScore','changeScore', 'assignGrade','read','write','checkStatus','setStatus']
RU=[{# a user can read his/her own scores in gradebooks for courses he/she has taken. 
     # change in and = into is
  'expressions':[{'eu':{ },'er':{'type':[('is','gradebook')],'crs':[('in','crsTaken')]}}],'operations':['readMyScores'],'decision':'allow'}, 
# a user (the instructor or TA) can add scores and read scores in the gradebook for courses he/she is teaching. 
 {'expressions':[{'eu':{ },'er':{'type':[('is','gradebook')],'crs':[('in','crsTaught')]}}],'operations':['addScore', 'readScore'],'decision':'allow'}, 
 # the instructor for a course (i.e., a faculty teaching the course) can change scores and assign grades in the gradebook for that course. 
{'expressions':[{'eu':{'position':[('is','faculty')]},'er':{'type':[('is','gradebook')],'crs':[('in','crsTaught')]}}],'operations':['changeScore','assignGrade'],'decision':'allow'}, 
 # a user in the registrar's office can read and modify all rosters.
{'expressions':[{'eu':{'udepartment':[('is','registrar')]},'er':{'type':[('is','roster')]}}],'operations':['read','write'],'decision':'allow'}, 
# the instructor for a course (i.e., a faculty teaching the course) can read the course roster
{'expressions':[{'eu':{'position':[('is','faculty')]},'er':{'type':[('is','roster')],'crs':[('in','crsTaught')]}}],'operations':['read'],'decision':'allow'}, 
# a user can read his/her own transcript.
{'expressions':[{'eu':{ },'er':{'type':[('is','transcript')],'student':[('=','uid')]}}],'operations':['read'],'decision':'allow'}, 
#the chair of a department can read the transcripts of all students in that department.
{'expressions':[{'eu':{'isChair':[('is',True)]},'er':{'type':[('is','transcript')],'rdepartment':[('=','udepartment')]}}],'operations':['read'],'decision':'allow'}, 
# a user in the registrar's office can read every student's transcript.
{'expressions':[{'eu':{'udepartment':[('is','registrar')]},'er':{'type':[('is','transcript')]}}],'operations':['read'],'decision':'allow'}, 
# a user can check the status of his/her own application 
{'expressions':[{'eu':{ },'er':{'type':[('is','application')],'student':[('=','uid')]}}],'operations':['checkStatus'],'decision':'allow'}, 
# a user in the admissions office can read, and update the status of, every application. 
{'expressions':[{'eu':{'udepartment':[('is','admissions')]},'er':{'type':[('is','application')]}}],'operations':['read','setStatus'],'decision':'allow'}
]
def dU(user,attribute):
  if attribute in user.keys():
    return user[attribute]
  return None

def dR(resource,attribute):
  if attribute in resource.keys():
    return resource[attribute]
  return None

ABAC_university=(U,R,O,UA,RA,dU,dR,RU)


In [3]:
def satisfied(u,r,expressions):
  for expression in expressions:
    eu=expression['eu']
    er=expression['er']
    #for all the keys in eu get the constrina of this key
    #if u does not satisfy this const return false
    for key in eu.keys():
      for (op,c) in eu[key]:
        #op,c=eu[key]
        #print(op,c)
        if op == 'is':
          if dU(u,key) != c:
            return False
        elif op == 'in':
          #if type(dU(u,c)!=list):
          #  return False
          if dU(u,c)==None:
            return False
          if type(dU(u,c)==list):
            if key not in dU(u,c):
               return False

    #for all the keys in er get the constrinat of the key
    for key in er.keys():
      for (op,c) in er[key]:
        #print(op,c)
        #op,c=er[key]
        if(dR(r,key)==None):
          return False
        if op =='is':
          if dR(r,key)!=c:
        #  print(dR(r,key),c)
            return False
        elif op == '=':
          if dR(r,key)!=dU(u,c) and dR(r,key)!=dR(r,c):
            return False
        elif op == 'in':
          if dU(u,c)==None and dR(r,c)==None:
            return False
          if type(dU(u,c)==list) and dU(u,c)!=None:
          #print(dU(u,c))
            if dR(r,key) not in dU(u,c):
              return False
          if type(dR(r,c)==list) and dR(r,c)!=None:
          #print(dU(u,c))
            if dR(r,key) not in dR(r,c):
              return False
  
  return True


def check_ABAC_policy(request,abac_policy):
  u,r,o =request
  _,_,_,_,_,_,_,rules = abac_policy
  for rule in rules:
    if o in rule['operations']:
      if satisfied(u,r,rule['expressions']):
        return True
  return False

# check the ABAC policy 
number_allow=0
number_deny=0
for u in U:
  for r in R:
    for o in O:
      if(check_ABAC_policy((u,r,o),ABAC_university)==True):
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'For ABAC policy:\nNumber of allow:{number_allow}\nNumber of deny:{number_deny}')

#print(f'{check_ABAC_policy((U[6],R[7],"readMyScores"),ABAC_university)}\n{U[6]}\n{R[7]}')
#print(f'{check_ABAC_policy((U[3],R[5],"addScore"),ABAC_university)}\n{U[3]}\n{R[5]}')
#print(f'Loss at epo {epoch}: {losses/len(training_data)}')



For ABAC policy:
Number of allow:56
Number of deny:1627


In [4]:
# Mordify the rules( resource constraint become independent)
import copy
RU_new=copy.deepcopy(RU)

attributes={'crs':['cs101','cs601','ee101','ee601'],'student':['csStu1','csStu2','applicant1'],'rdepartment':['registrar','admissions','ee','cs']}

#print(RU_new)
print("RU")
print(RU)

for rule in RU:

  user_cons=rule['expressions'][0]['eu']
  resource_cons=rule['expressions'][0]['er']
  for rc_key in resource_cons.keys():
    for (op,value) in resource_cons[rc_key]:
      if value in UA:
        RU_new.remove(rule)
        #print((op,value))
        for v in attributes[rc_key]:
          rule_new = copy.deepcopy(rule)
          if op == '=':
            if value not in rule_new['expressions'][0]['eu'].keys():
              rule_new['expressions'][0]['eu'][value]=[('is',v)]
            else:
              rule_new['expressions'][0]['eu'][value].append(('is',v)) 
              # add one uc
           # rule_new['expressions'][0]['er'][rc_key].append(('is',v)
          if op == 'in':
            if value not in rule_new['expressions'][0]['eu'].keys():
              rule_new['expressions'][0]['eu'][v]=[('in',value)]
            else:
              rule_new['expressions'][0]['eu'][v].append((op,value)) 
              # add one uc
          rule_new['expressions'][0]['er'][rc_key].append(('is',v))
          rule_new['expressions'][0]['er'][rc_key].remove((op,value))
          RU_new.append(rule_new)
for i in RU_new:
   print(i)


RU
[{'expressions': [{'eu': {}, 'er': {'type': [('is', 'gradebook')], 'crs': [('in', 'crsTaken')]}}], 'operations': ['readMyScores'], 'decision': 'allow'}, {'expressions': [{'eu': {}, 'er': {'type': [('is', 'gradebook')], 'crs': [('in', 'crsTaught')]}}], 'operations': ['addScore', 'readScore'], 'decision': 'allow'}, {'expressions': [{'eu': {'position': [('is', 'faculty')]}, 'er': {'type': [('is', 'gradebook')], 'crs': [('in', 'crsTaught')]}}], 'operations': ['changeScore', 'assignGrade'], 'decision': 'allow'}, {'expressions': [{'eu': {'udepartment': [('is', 'registrar')]}, 'er': {'type': [('is', 'roster')]}}], 'operations': ['read', 'write'], 'decision': 'allow'}, {'expressions': [{'eu': {'position': [('is', 'faculty')]}, 'er': {'type': [('is', 'roster')], 'crs': [('in', 'crsTaught')]}}], 'operations': ['read'], 'decision': 'allow'}, {'expressions': [{'eu': {}, 'er': {'type': [('is', 'transcript')], 'student': [('=', 'uid')]}}], 'operations': ['read'], 'decision': 'allow'}, {'expressio

In [5]:
def get_item_id(item,list):
  id = 0
  while(id<len(list)):
    if item == list[id]:
      return id
    id = id + 1
  return None

from numpy import *
# Calculate the number of user constraints
# Calculate the number of permissions(resource constraint,operation)
UC=[]
Permission=[]
Permission_set=[]
EU=[]
for rule in RU_new:
  user_cons=rule['expressions'][0]['eu']
#  if user_cons not in EU:
#    EU.append(user_cons)
  resource_cons=rule['expressions'][0]['er']
  operations=rule['operations']
  for uc_key in user_cons.keys():
    uc = {uc_key:user_cons[uc_key]}
    if uc not in UC:
      UC.append(uc)
 # if get_item_id(user_cons,EU) 

  if (resource_cons,operations) not in Permission:
    Permission.append((resource_cons,operations))
  if user_cons not in EU:
    EU.append(user_cons)
    Permission_set.append([(resource_cons,operations)])   
  elif user_cons in EU:
    Permission_set[get_item_id(user_cons,EU)].append((resource_cons,operations)) 
n_uc=len(UC)
n_p=len(Permission_set)
print(f'Number of user constraints:{n_uc}\nNumber of Permission :{len(Permission)}')
print(UC)
print(Permission)
print(f'Number of eu:{len(EU)}\nNumber of Permission sets:{n_p}')
print(EU)
print('\n')
for i in Permission_set:
  print(i)

Number of user constraints:17
Number of Permission :29
[{'udepartment': [('is', 'registrar')]}, {'udepartment': [('is', 'admissions')]}, {'cs101': [('in', 'crsTaken')]}, {'cs601': [('in', 'crsTaken')]}, {'ee101': [('in', 'crsTaken')]}, {'ee601': [('in', 'crsTaken')]}, {'cs101': [('in', 'crsTaught')]}, {'cs601': [('in', 'crsTaught')]}, {'ee101': [('in', 'crsTaught')]}, {'ee601': [('in', 'crsTaught')]}, {'position': [('is', 'faculty')]}, {'uid': [('is', 'csStu1')]}, {'uid': [('is', 'csStu2')]}, {'uid': [('is', 'applicant1')]}, {'isChair': [('is', True)]}, {'udepartment': [('is', 'ee')]}, {'udepartment': [('is', 'cs')]}]
[({'type': [('is', 'roster')]}, ['read', 'write']), ({'type': [('is', 'transcript')]}, ['read']), ({'type': [('is', 'application')]}, ['read', 'setStatus']), ({'type': [('is', 'gradebook')], 'crs': [('is', 'cs101')]}, ['readMyScores']), ({'type': [('is', 'gradebook')], 'crs': [('is', 'cs601')]}, ['readMyScores']), ({'type': [('is', 'gradebook')], 'crs': [('is', 'ee101')]}

In [6]:


def cover(ma,mb):# ma cover mb
  if ma.shape[1] != mb.shape[1]:
    return False
  for i in range(ma.shape[1]):
    if mb[0,i]==1 and ma[0,i]==0:
      return False


  return True

In [7]:
# permission-constraint matrix
pc=mat(zeros((n_p,n_uc)))
for rule in RU_new:
  user_cons=rule['expressions'][0]['eu']
  resource_cons=rule['expressions'][0]['er']
  operations=rule['operations']
  p_id=get_item_id(user_cons,EU)
  for uc_key in user_cons.keys():
    uc = {uc_key:user_cons[uc_key]}
    c_id=get_item_id(uc,UC)
    if(c_id==None):
      continue
    pc[p_id,c_id]=1
print(f'The pc matrix is\n {pc}')
rp=mat(diag(array(ones(n_p))))
for i in range(n_p-1):
  for j in range(i+1,n_p):
    if (cover(pc[i,],pc[j,])):
      rp[i,j]=1
    if (cover(pc[j,],pc[i,])):
      rp[j,i]=1

print(f'The rp matrix is\n {rp}')

The pc matrix is
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0.

In [9]:
def rest_of(l,r):
  vector = mat(zeros((1,n_uc)))
  for role in l:
    if role != r:
      vector = vector + rc[role,]
  for i in range(n_uc):
    if vector[0,i]>0:
      vector[0,i]=1
  return vector

import copy
# User role assignment 
n_r=n_p
rc=pc
URA = dict()
for u in U:
  URA[u['uid']]=[]
  uv=mat(ones((1,n_uc))) # user vector
  for cid in range(n_uc):
    expressions=[{'eu':UC[cid],'er':{}}]
    if satisfied(u,None,expressions)==False:
      uv[0,cid]=0
  for rid in range(n_r):
    if cover(uv,rc[rid,]):
      URA[u['uid']].append(rid)

# remove hierarchy
  for ri in URA[u['uid']]:
    rest = URA[u['uid']].copy()
    rest.remove(ri)

    for rj in rest:
      if cover(rc[rj,],rc[ri,]) and ri in URA[u['uid']]:
        URA[u['uid']].remove(ri)
#remove redundant
  for r in URA[u['uid']]:
    if len(URA[u['uid']])==1:
      break
    if cover(rest_of(URA[u['uid']],r),rc[r,]):
      URA[u['uid']].remove(r)

  ura =  URA[u['uid']]
  print(f'User {u}, uv {uv}, URA {ura}')


RPA = dict()
for r in range(n_p):
  RPA[r]=[]
for u in U:
  for pid in range(n_p):
    for o in O:
      for r in R:
        for role_id in URA[u['uid']]:
          if rp[role_id,pid]==1:
            #(er,operations) = Permission[pid]
            for (er,operations) in Permission_set[pid]:
              expressions=[{'eu':{},'er':er}]
              if satisfied(u,r,expressions) and o in operations:
                RPA[role_id].append((r,o))
                
print(f'The role-permission assignment is: {RPA}')


# generate the role-heirarchy
RH = []
for i in range(0,n_r-1):
  for j in range(i+1,n_r):
    if (cover(pc[i,],pc[j,])):
      RH.append((i,j))
    if (cover(pc[j,],pc[i,])):
      RH.append((j,i))    
print(f'The role-hierarchy relationship:  {RH}')

User {'uid': 'applicant1', 'position': 'applicant'}, uv [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]], URA [16]
User {'uid': 'csStu1', 'position': 'student', 'udepartment': 'cs', 'crsTaken': ['cs101'], 'crsTaught': ['cs601']}, uv [[0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1.]], URA [2, 7, 14]
User {'uid': 'csStu2', 'position': 'student', 'udepartment': 'cs', 'crsTaken': ['cs101', 'cs601']}, uv [[0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]], URA [2, 3, 15]
User {'uid': 'csFac1', 'position': 'faculty', 'udepartment': 'cs', 'crsTaught': ['cs101']}, uv [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]], URA [10]
User {'uid': 'csFac2', 'position': 'faculty', 'udepartment': 'cs', 'crsTaught': ['cs601']}, uv [[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1.]], URA [11]
User {'uid': 'csChair', 'isChair': True, 'udepartment': 'cs'}, uv [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]], URA [20]
User {'uid': 'eeFac1', 'position': 'faculty', 'udepartment': 'ee'

In [10]:
# check RBAC policy
def check_RBAC_policy(request,RPA,URA):
  u,r,o =request
  for role_id in URA[u['uid']]:
    if (r,o) in RPA[role_id]:
      return True


  return False

In [11]:

# check the RBAC policy 
number_allow=0
number_deny=0
for u in U:
  for r in R:
    for o in O:
      if(check_RBAC_policy((u,r,o),RPA,URA)==True):
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'For RBAC:\nNumber of allow:{number_allow}\nNumber of deny:{number_deny}')

# check the ABAC policy 
number_allow=0
number_deny=0
for u in U:
  for r in R:
    for o in O:
      if(check_ABAC_policy((u,r,o),ABAC_university)==True):
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'For ABAC:\nNumber of allow:{number_allow}\nNumber of deny:{number_deny}')


#number_allow=0
#number_deny=0

correct=0
for u in U:
  for r in R:
    for o in O:
      if (check_ABAC_policy((u,r,o),ABAC_university)==True and check_RBAC_policy((u,r,o),RPA,URA)==False):
        print(f'TF{u} {r} {o}')
      elif (check_ABAC_policy((u,r,o),ABAC_university)==False and check_RBAC_policy((u,r,o),RPA,URA)==True):
        print(f'FT{u} {r} {o}')
        #number_allow= number_allow+1
      elif (check_ABAC_policy((u,r,o),ABAC_university)==True and check_RBAC_policy((u,r,o),RPA,URA)==True):
        correct = correct +1
      elif (check_ABAC_policy((u,r,o),ABAC_university)==False and check_RBAC_policy((u,r,o),RPA,URA)==False):
        correct = correct +1
        #number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'For number of correct:{correct}')

For RBAC:
Number of allow:56
Number of deny:1627
For ABAC:
Number of allow:56
Number of deny:1627
For number of correct:1683
